#### Import Term Use Cases

The SAS Glossary API allows users to create terms by importing them from a CSV file.  The use cases provided here demonstrate how to import terms from a CSV file by creating an import job, monitoring its progress, and retrieving the log file that contains the results.

##### **Imports and Global Variables** <a id='imports'></a>
Run this cell before any of the others. It imports the packages and sets the variables that will be used throughout the notebook.

In [ ]:
import requests
import json
import time
sasserver = "https://your_server"
print (sasserver)
prefix = "Sample"

##### Get Access token <a id='authentication'></a>
This use case sets the access_token that will be used in all the calls in this use case. Edit the payload object with a username and password. Run this use case as needed because the token will expire after a certain amount of time. 

In [ ]:
url = sasserver + "/SASLogon/oauth/token#password"

payload={
    'grant_type': 'password',
    'username': 'your_username',
    'password': 'your_password'
}
files=[

]
headers = {
  'Authorization': 'Basic c2FzLmVjOg==',
}

response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)

if response.status_code == 200:
    print("Successfully authenticated.")
    response = response.json()
    access_token = response["access_token"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")
 

##### Create a Term Type
This use case creates a term type that will be used in the following examples. Specify an appropriate name, label, and description. You also can specify custom attributes, if they are necessary.

In [ ]:

url = sasserver  + "/glossary/termTypes"

payload = json.dumps({
  "name": prefix + " Term Type",
  "description": "Sample Term Type with all attribute values populated",
  "label": "Sample Term Type",
  "attributes": [
    {
      "name": "singleLineTextAttribute",
      "label": "A single-line text attribute",
      "description": "This is a description of the attribute. Single-line text attributes can be up to 4000 characters long and cannot contain newline characters.",
      "required": True,
      "type": "single-line",
      "defaultValue": "This is an example of a single-line text attribute."
    },
    {
      "name": "multiLineTextAttribute",
      "label": "A multi-line text attribute",
      "description": "Multi-line text attributes can be up to 4000 characters.  Newline characters are allowed.",
      "type": "multi-line",
      "defaultValue": "This is an example of a multi-line text attribute\\nwith\\nnewline\\ncharacters."
    },
    {
      "name": "booleanAttribute",
      "label": "A boolean attribute",
      "description": "Boolean attributes can be either \"true\" or \"false\"",
      "type": "boolean",
      "defaultValue": "true"
    },
    {
      "name": "dateAttribute",
      "label": "A date attribute",
      "description": "Date attributes should be specified with the format \"YYYY-MM-DD\"",
      "type": "date"
    },
    {
      "name": "timeAttribute",
      "label": "A time attribute",
      "description": "Time attributes should be specified with the format \"hh:mm:ssZ\"",
      "type": "time",
      "defaultValue": "10:23:45Z"
    },
    {
      "name": "dateTimeAttribute",
      "label": "A date-time attribute",
      "description": "Date-time attributes should be specified with the format \"YYYY-MM-DDThh:mm:ssZ\"",
      "type": "date-time",
      "defaultValue": "2024-02-12T10:23:45.000Z"
    },
    {
      "name": "singleSelectAttribute",
      "label": "A single-select attribute",
      "description": "Single-select attributes must specify at least one item in the \"items\" field",
      "type": "single-select",
      "items": [
        "Item 1",
        "Item 2",
        "Item 3"
      ],
      "defaultValue": "Item 1"
    }
  ]
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term.type+json',
  'Content-Type': 'application/vnd.sas.glossary.term.type+json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print(f"Successfully created term type.\n{json.dumps(response.json(), indent=2)}")
    response = response.json()
    termTypeId = response["id"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")


##### Import CSV
This use case creates an import job to import a term CSV file.

In [ ]:
url = sasserver + "/glossary/importTerms"

payload = {'updateExisting': 'true'}
files = [
    ('termCSV', ('import.csv', open('../../../import.csv', 'rb'), 'text/csv'))

]
headers = {
  'Accept': 'application/vnd.sas.job.execution.job+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)

if response.status_code == 202:
    print("Successfully retrieved file.")
    response = response.json()
    print(json.dumps(response, indent=2))
    jobId = response["id"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Get Import Job
This use case retrieves details and status information about a specific term import job.

In [ ]:
url = sasserver + "/jobExecution/jobs/" + jobId

payload = {}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + access_token
}

maxRetries = 20
sleepBetweenTries = 5


for i in range(maxRetries):
    response = requests.request("GET", url, headers=headers, data=payload, verify=False)
    if response.status_code == 200:
        response = response.json()
        print(f"Job State: {response['state']}")
        if response['state'] == 'completed':
            logLocation = response["logLocation"]
            print(f"logLocation: {logLocation}")
            break
    else:
        print('Error retrieving import job:', response.text)

        print(f"Request {i+1}: Status code: {response.status_code}")

    time.sleep(sleepBetweenTries)

    

##### Get Import Log
This use case retrieves the record of activities and details related to a specific term import job.

In [ ]:
url = sasserver + logLocation + "/content"

payload = {}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved import job log.")
    print(response.text)
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Clean Up
This use case deletes terms and term types that were created by these examples.

In [ ]:
# Getting All Terms
url = sasserver + "/catalog/instances?filter=startsWith(name," + prefix + ")&limit=20&sortBy=creationTimeStamp:descending"

headers = {
  'Accept': 'application/vnd.sas.collection+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    instances = response.json().get('items', [])

    host = sasserver
    token = access_token

    # Deleting Terms
    for index, a_term in enumerate(instances):
        del_link = next(link['href'] for link in a_term['links'] if link['rel'] == 'delete')

        del_request_url = host + del_link
        del_request_headers = {'Authorization': 'Bearer ' + token}

        response = requests.delete(del_request_url, headers=del_request_headers, verify=False)
        if response.status_code == 204:
            print('Deleted term with id:', a_term['id'])
        else:
            print('Error deleting term:', response.text)

    print('All terms have been processed.')

else:
    print(f"Error retrieving terms:\n{json.dumps(response.json(), indent=2)}")

# Deleting Term Types
url = sasserver + "/glossary/termTypes/" + termTypeId

payload = {}
headers = {
  'Accept': 'application/vnd.sas.glossary.term.type+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)

if response.status_code == 204:
    print("Successfully deleted term type.")
    
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")